# Setting up inputs for the gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-10  
  
In this notebook, we will prepare all necessary inputs need to run the gridded version of DBPM. Note that this step needs to be completed only once for each model run.  

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
import xarray as xr
import json
import pandas as pd
import numpy as np
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

## Defining output folder

In [2]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/'
out_folder = os.path.join(base_folder, 'gridded_params')
os.makedirs(out_folder, exist_ok = True)

## Transforming DBPM parameters to Python-friendly format
These parameters are the outputs of the `sizeparam` function for R. They were calculated and stored in the script ([04_calculating_dbpm_fishing_params](new_workflow/04_calculating_dbpm_fishing_params.R)) as a `json` file.

In [3]:
#Loading gridded parameters
gridded_params = json.load(open(
    os.path.join(out_folder, 'dbpm_gridded_size_params_fao-58.json')))

#Transforming them to Python friendly format
gridded_python = uf.gridded_param_python(gridded_params)

### Adding useful entries to gridded parameters
These variables are used multiple times within the `gridded_size_model` script, so they will be added for convenience.

In [4]:
# Size classes
log10_size_bins = np.array(gridded_python['log10_size_bins'])
# Minimum (log10) size class for predators
log10_ind_min_pred_size = log10_size_bins[gridded_python['ind_min_pred_size']]
ind_min_detritivore_size = gridded_python['ind_min_detritivore_size']
log10_ind_min_detritivore_size = log10_size_bins[ind_min_detritivore_size]
log10_ind_min_fish_pred = log10_size_bins[gridded_python['ind_min_fish_pred']]
log10_ind_min_fish_det = log10_size_bins[gridded_python['ind_min_fish_det']]

#Adding new variables to gridded parameters
gridded_python['log10_ind_min_pred_size'] = log10_ind_min_pred_size
gridded_python['log10_ind_min_detritivore_size'] = log10_ind_min_detritivore_size

### Saving gridded parameters

In [80]:
out_gridded = os.path.join(out_folder, 'dbpm_gridded_size_params_fao-58_python.json')

#Save to disk
with open(out_gridded, 'w') as outfile:
    json.dump(gridded_python, outfile)

## Loading gridded input data
Gridded data are needed both as direct input to DBPM and to calculate params, such as habitat preference, pelagic predator and benthic detritivore size spectrum, among other things.

In [5]:
depth = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_deptho_15arcmin_fao-58_fixed.zarr/'))['deptho']

In [37]:
#Loading intercept data for spinup period
int_phy_zoo_spinup = xr.open_zarr(
    os.path.join(base_folder, 'gridded',
                 'gfdl-mom6-cobalt2_spinup_intercept_15arcmin_fao-58_monthly_1841_1960.zarr/'))['intercept']

#Loading intercept data for model period and fixing time values
int_phy_zoo  = xr.open_zarr(
    os.path.join(base_folder, 'gridded',
                 'gfdl-mom6-cobalt2_obsclim_intercept_15arcmin_fao-58_monthly_1961_2010.zarr/'))['intercept']
int_phy_zoo['time'] = pd.DatetimeIndex(int_phy_zoo.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

/jobfs/132307763.gadi-pbs/ipykernel_316457/1484338636.py:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  int_phy_zoo['time'] = pd.DatetimeIndex(int_phy_zoo.indexes['time'].to_datetimeindex(),


In [39]:
#Loading slope data for spinup period
slope_phy_zoo_spinup = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_spinup_slope_15arcmin_fao-58_monthly_1841_1960.zarr/'))['slope']

#Loading slope data for model period and fixing time values
slope_phy_zoo  = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_slope_15arcmin_fao-58_monthly_1961_2010.zarr/'))['slope']
slope_phy_zoo['time'] = pd.DatetimeIndex(slope_phy_zoo.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

/jobfs/132307763.gadi-pbs/ipykernel_316457/361179567.py:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  slope_phy_zoo['time'] = pd.DatetimeIndex(slope_phy_zoo.indexes['time'].to_datetimeindex(),


In [10]:
#Loading sea surface temperature data for spinup period
sea_surf_temp_spinup = xr.open_zarr(
    os.path.join(base_folder, 'gridded',
                 'gfdl-mom6-cobalt2_spinup_tos_15arcmin_fao-58_monthly_1841_1960.zarr'))['tos']

#Loading sea surface temperature data for model period and fixing time values
sea_surf_temp  = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_tos_15arcmin_fao-58_monthly_1961_2010.zarr/'))['tos']
sea_surf_temp['time'] = pd.DatetimeIndex(sea_surf_temp.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

/jobfs/132307763.gadi-pbs/ipykernel_316457/505148546.py:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  sea_surf_temp['time'] = pd.DatetimeIndex(sea_surf_temp.indexes['time'].to_datetimeindex(),


In [9]:
#Loading bottom ocean temperature data for spinup period
sea_floor_temp_spinup = xr.open_zarr(
    os.path.join(base_folder, 'gridded',
                 'gfdl-mom6-cobalt2_spinup_tob_15arcmin_fao-58_monthly_1841_1960.zarr'))['tob']

#Loading bottom ocean temperature data for model period and fixing time values
sea_floor_temp  = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_tob_15arcmin_fao-58_monthly_1961_2010.zarr/'))['tob']
sea_floor_temp['time'] = pd.DatetimeIndex(sea_floor_temp.indexes['time'].to_datetimeindex(),
                                          dtype = 'datetime64[ns]')

/jobfs/132307763.gadi-pbs/ipykernel_304759/2851380992.py:10: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  sea_floor_temp['time'] = pd.DatetimeIndex(sea_floor_temp.indexes['time'].to_datetimeindex(),


### Creating time variable that contains a timestep for initialisation of DBPM
This will only be applied to the `spinup` period. It will add a timestep before the start of the period to initialise DBPM. This is not applied to the modelling period because values from the last time step of the `spinup` period will be used to run DBPM.

In [8]:
time_spinup = (np.append((int_phy_zoo_spinup.time.min().values-np.timedelta64(1, 'm')),
                         int_phy_zoo_spinup.time.values))

## Creating gridded variables needed for DBPM run
We will create data array versions of the gridded inputs.

### DBPM size classes

In [11]:
# Log10 size of individuals found in the model
log10_size_bins_mat = xr.DataArray(data = log10_size_bins,
                                   dims = ['size_class'], 
                                   coords = {'size_class': log10_size_bins})

# size of individuals found in the model
size_bin_vals = 10**log10_size_bins_mat

### Fishing effort
Effort data for the `spinup` and modelling period is provided in a single entry in `gridded_python`. We will split this data into the `spinup` and modelling period.

In [12]:
effort_spinup = (xr.DataArray(gridded_python['effort'][:len(int_phy_zoo_spinup.time)], 
                              dims = 'time', 
                              coords = {'time': int_phy_zoo_spinup.time}).
                 expand_dims({'lat': depth.lat, 'lon': depth.lon}).
                 transpose('time', 'lat', 'lon'))

effort = (xr.DataArray(gridded_python['effort'][len(int_phy_zoo_spinup.time):],
                       dims = 'time', coords = {'time': int_phy_zoo.time}).
          expand_dims({'lat': depth.lat, 'lon': depth.lon}).
          transpose('time', 'lat', 'lon'))

### Setting habitat preferences
Predator coupling to benthic prey, which is depth dependent, 0.75 above 500 m, 0.5 between 500 and 1800 m, and 0 below 1800 m. Predator coupling to pelagic prey is equal to `1-(benthic coupling)`.  
  
These values were suggested by Clive Trueman based on stable isotope work, and proportion of biomass. Rockall Trough studies.

In [11]:
pref_benthos = (0.8*np.exp((-1/250*depth)))
pref_pelagic = (1-pref_benthos)

### Initiliasing values for pelagic predators and benthic detritivores

In [10]:
init_pred = (xr.DataArray(data = gridded_python['init_pred'], 
                          dims = ['size_class'], 
                          coords = {'size_class': log10_size_bins}).
             expand_dims({'lat': depth.lat, 'lon': depth.lon}))

init_detritivores = (xr.DataArray(data = gridded_python['init_detritivores'], 
                                  dims = ['size_class'], 
                                  coords = {'size_class': log10_size_bins}).
                     expand_dims({'lat': depth.lat, 'lon': depth.lon}))

### Calculating constant growth and mortality

In [19]:
pred_prey_mat = uf.pred_prey_matrix(log10_size_bins)

constant_growth = xr.DataArray(uf.gphi_f(pred_prey_mat, 
                                         gridded_python['log10_pred_prey_ratio'],
                                         gridded_python['log_prey_pref']),
                               dims = ['size_class', 'sc'])

constant_mortality = xr.DataArray(uf.mphi_f(-pred_prey_mat, 
                                            gridded_python['log10_pred_prey_ratio'],
                                            gridded_python['log_prey_pref'],
                                            gridded_python['metabolic_req_pred']),
                                  dims = ['size_class', 'sc'])

### Building metabolic requirements lookup table 
Metabolic requirements are estimated per size class

In [20]:
met_req_log10_size_bins = uf.expax_f(log10_size_bins_mat, 
                                     gridded_python['metabolic_req_pred'])

In [21]:
consume_pelagic = (pref_pelagic*gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)
consume_benthos = (pref_benthos*gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)

### Creating a gridded time series of intercept of plankton size spectrum 
The size spectrum was estimated by the `GetPPIntSlope` function using GFDL-MOM6-COBALT2 outputs in script ([01_processing_dbpm_global_inputs](new_workflow/01_processing_dbpm_global_inputs.ipynb)).

In [41]:
ui0_spinup = 10**int_phy_zoo_spinup
ui0 = 10**int_phy_zoo

### Calculating `predator` biomass for `spinup` and modelling period
Values are derived from plankton-zooplankton size spectrum, slope and size class.

In [42]:
# Calculating predator biomass for spinup period
pred_start_spinup = ui0_spinup*(10**(slope_phy_zoo_spinup*log10_size_bins_mat))
# Keep values outside the predator size classes, otherwise change to zero (0)
pred_start_spinup = xr.where(pred_start_spinup.size_class <= log10_ind_min_pred_size,
                             pred_start_spinup, 0)
# Copy first time step predator values as 'initialisation' timestep
pred_start_spinup = xr.concat([pred_start_spinup.isel(time = 0), pred_start_spinup],
                              dim = 'time')
# Update time labels
pred_start_spinup['time'] = time_spinup
# Assigning initial values (time = 0) for predators from gridded parameters
predators_spinup = xr.where((pred_start_spinup.time == pred_start_spinup.time.min()) & 
                            (pred_start_spinup.size_class >= log10_ind_min_pred_size),
                            init_pred, pred_start_spinup)

# Calculating predator biomass for modelling period
pred_start = ui0*(10**(slope_phy_zoo*log10_size_bins_mat))
# Keep values outside the predator size classes, otherwise change to zero (0)
predators = xr.where(pred_start.size_class <= log10_ind_min_pred_size,
                      pred_start, 0)

### Detritivore biomass and detritus
Creating gridded versions of `detritivores` and `detritus` initalising values to be used in gridded DBPM.

In [63]:
# Assigning initial values (time = 0) for detritivores from gridded parameters
detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size),
                        init_detritivores, 0)
# Adding time dimension
detritivores = detritivores.expand_dims({'time': [min(time_spinup)]})

# Initialising detritus
detritus = xr.zeros_like(detritivores.isel(size_class = 0).drop_vars('size_class'))
# Assigning initial values (time = 0) for detritus from gridded parameters
detritus = xr.where(detritus == 0, gridded_python['init_detritus'], detritus)

### Note
If no initial values for `predators` and `detritivores` are included in the gridded parameters, use the following code instead:  
  
```
predators_spinup = xr.where((pred_start_spinup.time == pred_start_spinup.time.min()) &
                       (pred_start_spinup.size_class >= log10_ind_min_pred_size) &
                       (pred_start_spinup.size_class < log10_size_bins[120]),
                       init_pred, pred_start_spinup)

detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size) &
                         (init_detritivores.size_class < log10_size_bins[120]), 
                         init_detritivores, detri_init)
```

### Fishing mortality
Here `fishing_mort_pred` and `fishing_mort_det` are fixed catchability terms for `predators` and `detritivores`. Fishing mortality is estimated only for individuals within fished sizes (`ind_min_fish_pred` or `ind_min_fish_det`).

In [32]:
# Using fish_mort_pred as a base for fishing mortality for predators
fish_mort_pred = xr.DataArray(
    np.repeat(gridded_python['fish_mort_pred'], 
              np.prod(consume_pelagic.shape)).reshape(consume_pelagic.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_pelagic.lat, 'lon': consume_pelagic.lon,
             'size_class': consume_pelagic.size_class})

# Changing mortality to zero outside the predator size fished
fishing_mort_pred = xr.where((fish_mort_pred.size_class >= log10_ind_min_fish_pred) &
                          (fish_mort_pred.size_class < 
                           fish_mort_pred.size_class.max()),
                          fish_mort_pred, 0)

In [33]:
# Using fish_mort_det as a base for fishing mortality for detritivores
fish_mort_det = xr.DataArray(
    np.repeat(gridded_python['fish_mort_detritivore'], 
              np.prod(consume_benthos.shape)).reshape(consume_benthos.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_benthos.lat, 'lon': consume_benthos.lon,
             'size_class': consume_benthos.size_class})

# Changing mortality to zero outside the detritivore size fished
fishing_mort_det = xr.where((fish_mort_det.size_class >= log10_ind_min_fish_det) &
                          (fish_mort_det.size_class < 
                           fish_mort_det.size_class.max()),
                          fish_mort_det, 0)

### Temperature effect on pelagic organisms
This will be calculated for the `spinup` and modelling periods

In [34]:
pel_tempeffect_spinup = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                               (gridded_python['boltzmann']*(sea_surf_temp_spinup+273)))

pel_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_surf_temp+273)))

### Temperature effect on benthic organisms
This will be calculated for the `spinup` and modelling periods

In [35]:
ben_tempeffect_spinup = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                               (gridded_python['boltzmann']*(sea_floor_temp_spinup+273)))

ben_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_floor_temp+273)))

### Note
If `temp_effect` is not needed, then simply assign `1` to `pel_tempeffect` and `ben_tempeffect`:  
  
```
pel_tempeffect = 1
ben_tempeffect = 1
```

## Saving gridded inputs
We will now save all the gridded inputs needed to run the DBPM model.

In [36]:
log10_size_bins_mat.name = 'size_bins'
log10_size_bins_mat.to_zarr('outputs/log10_size_bins_matrix.zarr/',
                            consolidated = True, mode = 'w')

In [44]:
effort_spinup.name = 'effort'
effort_out = effort_spinup.chunk({'time': 120})
effort_out.to_zarr(
    os.path.join(out_folder, 'effort_spinup_15arcmin_fao-58_monthly_1841_1960.zarr/'),
                   consolidated = True, mode = 'w')

effort.name = 'effort'
effort_out = effort.chunk({'time': 120})
effort_out.to_zarr(
    os.path.join(out_folder, 'effort_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                   consolidated = True, mode = 'w')

In [45]:
pref_benthos.name = 'pref_benthos'
pref_benthos.to_zarr(os.path.join(out_folder, 
                                  'pref-benthos_15arcmin_fao-58_fixed.zarr/'),
                     consolidated = True, mode = 'w')
pref_pelagic.name = 'pref_pelagic'
pref_pelagic.to_zarr(os.path.join(out_folder, 
                                  'pref-pelagic_15arcmin_fao-58_fixed.zarr/'), 
                     consolidated = True, mode = 'w')

In [46]:
constant_growth.name = 'constant_growth'
constant_growth.to_zarr(os.path.join(out_folder,
                                     'const-growth_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

constant_mortality.name = 'constant_mortality'
constant_mortality.to_zarr(os.path.join(out_folder,
                                     'const-mort_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [47]:
consume_pelagic.name = 'consume_pelagic'
consume_pelagic.to_zarr(os.path.join(out_folder,
                                     'consume-pelagic_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

consume_benthos.name = 'consume_benthos'
consume_benthos.to_zarr(os.path.join(out_folder,
                                     'consume-benthos_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

Dividing predator data into decadal files before saving them.

In [25]:
predators_spinup.name = 'predators'

yrs = predators_spinup.time.dt.year
dec = ((yrs - yrs[0])/10).astype(int)

predators_spinup = predators_spinup.assign_coords({'dec': dec})

for i, da in predators_spinup.groupby('dec'):
    ymin = str(da.time.dt.year.min().values)
    ymax = str(da.time.dt.year.max().values)
    da_out = da.drop_vars('dec')
    da_out.to_zarr(
        os.path.join(out_folder, 
                     f'predators_spinup_15arcmin_fao-58_monthly_{ymin}_{ymax}.zarr/'),
        consolidated = True, mode = 'w')

In [46]:
predators.name = 'predators'

yrs = predators.time.dt.year
dec = ((yrs - yrs[0])/10).astype(int)

predators = predators.assign_coords({'dec': dec})

for i, da in predators.groupby('dec'):
    ymin = str(da.time.dt.year.min().values)
    ymax = str(da.time.dt.year.max().values)
    da_out = da.drop_vars('dec')
    da_out.to_zarr(
        os.path.join(out_folder, f'predators_15arcmin_fao-58_monthly_{ymin}_{ymax}.zarr/'),
        consolidated = True, mode = 'w')

In [68]:
detritivores.name = 'detritivores'
detritivores.to_zarr(os.path.join(out_folder, 
                                  'detritivores_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                     consolidated = True, mode = 'w')

detritus.name = 'detritus'
detritus.to_zarr(os.path.join(out_folder, 
                              'detritus_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                 consolidated = True, mode = 'w')

In [58]:
fishing_mort_pred.name = 'fish_mort_pred'
fishing_mort_pred.to_zarr(os.path.join(out_folder, 
                                       'fish-mort-pred_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')

fishing_mort_det.name = 'fish_mort_det'
fishing_mort_det.to_zarr(os.path.join(
    out_folder, 'fish-mort-det_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                         consolidated = True, mode = 'w')

In [72]:
pel_tempeffect_spinup.name = 'pel_temp_eff' 
pel_tempeffect_out = pel_tempeffect_spinup.chunk({'lat': 106, 'lon': 480})
pel_tempeffect_out.to_zarr(
    os.path.join(out_folder, 
                 'pel-temp-eff_spinup_15arcmin_fao-58_monthly_1841_1960.zarr/'),
    consolidated = True, mode = 'w')

pel_tempeffect.name = 'pel_temp_eff' 
pel_tempeffect_out = pel_tempeffect.chunk({'lat': 106, 'lon': 480})
pel_tempeffect_out.to_zarr(
    os.path.join(out_folder, 'pel-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')

In [75]:
ben_tempeffect_spinup.name = 'ben_temp_eff'
ben_tempeffect_out = ben_tempeffect_spinup.chunk({'lat': 106, 'lon': 480})
ben_tempeffect_out.to_zarr(
    os.path.join(out_folder, 
                 'ben-temp-eff_spinup_15arcmin_fao-58_monthly_1841_1960.zarr/'),
    consolidated = True, mode = 'w')

ben_tempeffect.name = 'ben_temp_eff'
ben_tempeffect_out = ben_tempeffect.chunk({'lat': 106, 'lon': 480})
ben_tempeffect_out.to_zarr(
    os.path.join(out_folder, 'ben-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')